<a href="https://colab.research.google.com/github/simecek/dspracticum2024/blob/main/lesson02/FashionMNIST_Dense.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [203]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [204]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


**2. Data Preparation**

In [205]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

**3. Neural Network Model**

In [206]:
class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        logits = self.fc5(x)
        return logits

model = DenseNet().to(device)
model

DenseNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [207]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

**5. Training loop**

In [208]:
for epoch in range(50):  # Train for 50 epochs
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/50], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/50], Loss: 1.4397
Epoch [2/50], Loss: 0.7569
Epoch [3/50], Loss: 0.5543
Epoch [4/50], Loss: 0.4779
Epoch [5/50], Loss: 0.4342
Epoch [6/50], Loss: 0.4034
Epoch [7/50], Loss: 0.3820
Epoch [8/50], Loss: 0.3636
Epoch [9/50], Loss: 0.3522
Epoch [10/50], Loss: 0.3367
Epoch [11/50], Loss: 0.3258
Epoch [12/50], Loss: 0.3212
Epoch [13/50], Loss: 0.3105
Epoch [14/50], Loss: 0.3044
Epoch [15/50], Loss: 0.2970
Epoch [16/50], Loss: 0.2914
Epoch [17/50], Loss: 0.2850
Epoch [18/50], Loss: 0.2779
Epoch [19/50], Loss: 0.2737
Epoch [20/50], Loss: 0.2686
Epoch [21/50], Loss: 0.2648
Epoch [22/50], Loss: 0.2589
Epoch [23/50], Loss: 0.2564
Epoch [24/50], Loss: 0.2511
Epoch [25/50], Loss: 0.2474
Epoch [26/50], Loss: 0.2437
Epoch [27/50], Loss: 0.2400
Epoch [28/50], Loss: 0.2363
Epoch [29/50], Loss: 0.2324
Epoch [30/50], Loss: 0.2329
Epoch [31/50], Loss: 0.2291
Epoch [32/50], Loss: 0.2270
Epoch [33/50], Loss: 0.2231
Epoch [34/50], Loss: 0.2194
Epoch [35/50], Loss: 0.2156
Epoch [36/50], Loss: 0.2148
E

**6. Evaluation on the test set**

In [209]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 87.27%
